# Assignment 3 Softmax
We underestimated how hard it was going from logistic regression to softmax classification. The main problem is to get the weights structure to interact the right way with the rest of the code as it is now a (785,10) tupple instead of (785,1).
We have gone through the code and written pseudocode for what we need to change to get it working, but didn't have time to implement.

The one-hot-encoding works as expected.

### loss_regularized()
The loss function needs to change to equation (8) in the assignment text. The rest can remain unchanged since j(w) = E(w) + lambda*C(w) still.

### forward_pass()
The forward pass now has a new way of propagating described by the softmax function in equation (6)

### the rest
The rest of the code should not need changing except doublechecking the new variable-dimensions. The gradient decent expression remains unchanged.

In [1]:
import numpy as np
import mnist
import matplotlib.pyplot as plt # Plotting library

#mnist.init() #Run only if you do not have the data downloaded
X_train, Y_train, X_test, Y_test = mnist.load()


In [2]:
#Splitting into smaller development sets
X_train_orig = X_train[:20000]
Y_train_orig = Y_train[:20000]
X_test_orig = X_test[8000:]
Y_test_orig = Y_test[8000:]


#Bias trick on every picture
X_train_orig = np.insert(X_train,784,1, axis=1)
X_test_orig = np.insert(X_test,784,1,axis=1)

In [3]:
#Shaping the data doing One-hot-encoding

X_train = X_train_orig
Y_train = np.zeros((Y_train_orig.size,10))
X_test = X_test_orig
Y_test = np.zeros((Y_test_orig.size,10))


for i in range(Y_train_orig.size):
    for j in range(10):
        if j == Y_train_orig[i]:
            Y_train[i,j] = 1



#Splitting train into train and validation
end = Y_train.size
start = int(Y_train.size * 0.9)

X_val = X_train[start:end]
Y_val = Y_train[start:end]

X_train = X_train[:start]
Y_train = Y_train[:start]

#print(Y_train)


In [4]:
# Hyperparameters
epochs = 500
batch_size = 50
learning_rate_0 = 0.00001
learning_rate = 0.0000001
T = 10
regularization_lambda = 0.01
classes = 10

# Tracking variables
TRAIN_LOSS = []
VAL_LOSS = []
TEST_LOSS = []
TRAINING_STEP = []
num_features = X_train.shape[1]

TRAIN_CORRECT_PERCENT = []
TEST_CORRECT_PERCENT = []
VAL_CORRECT_PERCENT = []


num_batches_per_epoch = X_train.shape[0] // batch_size
print(num_batches_per_epoch)
check_step = num_batches_per_epoch // 10 # How often we should calculate validation / train loss

1200


In [5]:
#The cross entropy loss function E(w)
def loss_regularized(targets, outputs, weights, param):
    assert targets.shape == outputs.shape
    #the expression for entropy_loss is the only on that needs changing as J(w)=E(w) + lambda*C(w) still
    #for all pixels, for all classes: targets*np.log(outputs)
    entropy_loss = weights*targets*np.log(outputs)
    mean_entropy_loss = -entropy_loss.mean()
    
    weights= np.reshape(weights,weights.size)
    regularized_loss = mean_entropy_loss +param*np.dot(weights,weights)

    return regularized_loss

In [6]:
def forward_pass(X, w):
    #softmax
    #instead of the old logistic regression pass we now use softmax function here
    a = np.zeros()
    for k in range(classes):
        a[k] = w[:,k]*X
        #output = np.e**(a[k]) / sum of np.e**(a[k']) for all k'

    return output

In [7]:
def gradient_decent_regularized(X, outputs, targets, weights, learning_rate, param):
  N = X.shape[0]
  assert outputs.shape == targets.shape

  #As long as weights has the right shape nothing should need changing here
  dw = X * (targets-outputs) + np.transpose(2*param*weights) 
  dw = dw.mean(axis = 0).reshape(-1, 1) # Normalize gradient w.r.t number of training samples
  assert dw.shape == weights.shape, "dw: {}, w: {}".format(dw.shape, weights.shape)
  weights = weights + learning_rate * dw

  return weights

In [8]:
def early_stop(VAL_LOSS):
    if VAL_LOSS[len(VAL_LOSS)-1]>VAL_LOSS[len(VAL_LOSS)-2]:
        return True
    return False

In [9]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [10]:
def anneal_learning_rate(training_it):
    alpha_t = learning_rate_0/(1+(training_it/T))
    return alpha_t

In [11]:
#The weight structure now has a different structure to reflect that we have 10 classes
#Getting the w-parameter to work correctly with vectorized logic was to hard to do in little time
w = np.zeros((num_features, 10))

def train_loop(w):
  #print(w.shape)
  training_it = 0
  stop_condition_counter = 0
  for epoch in range(epochs):
    #shuffle(X_train, Y_train)
    X_train_c, Y_train_c = unison_shuffled_copies(X_train, Y_train)
    for i in range(num_batches_per_epoch):
      
      X_batch = X_train_c[i*batch_size:(i+1)*batch_size]
      Y_batch = Y_train_c[i*batch_size:(i+1)*batch_size]

      out = forward_pass(X_batch, w)
      learning_rate = anneal_learning_rate(training_it)
      w = gradient_decent(X_batch, out, Y_batch, w, learning_rate)
      #print(out)
      training_it += 1
      if i % check_step == 0:
        
        # Training set
        TRAINING_STEP.append(training_it)
        
        train_out = forward_pass(X_train, w)
        train_loss = loss_regularized(Y_train, train_out)
        TRAIN_LOSS.append(train_loss)
        
        val_out = forward_pass(X_val, w)
        val_loss = loss_regularized(Y_val, val_out)
        VAL_LOSS.append(val_loss)
        
        test_out = forward_pass(X_test, w)
        test_loss = loss_regularized(Y_test, test_out)
        TEST_LOSS.append(test_loss)
        
        #plotting percentage of correct guesses
        train_correct = 0
        for i in range(Y_train.size):
            if (Y_train[i] == 1 and train_out[i] >= 0.5) or (Y_train[i] == 0 and train_out[i] < 0.5):
                train_correct +=1
        TRAIN_CORRECT_PERCENT.append((train_correct / Y_train.size)*100)
        
        test_correct = 0
        for i in range(Y_test.size):
            if (Y_test[i] == 1 and test_out[i] >= 0.5) or (Y_test[i] == 0 and test_out[i] < 0.5):
                test_correct +=1
        TEST_CORRECT_PERCENT.append((test_correct / Y_test.size)*100)
        
        val_correct = 0
        for i in range(Y_val.size):
            if (Y_val[i] == 1 and val_out[i] >= 0.5) or (Y_val[i] == 0 and val_out[i] < 0.5):
                val_correct +=1
        VAL_CORRECT_PERCENT.append((val_correct / Y_val.size)*100)
        
        
        if early_stop(VAL_LOSS):
            stop_condition_counter += 1
            if stop_condition_counter > 3:
                print("Early stopping Triggered!")
                print(training_it)
                return w
        else:
            stop_condition_counter = 0
            

  return w

#w = train_loop(w)
    
